<a href="https://colab.research.google.com/github/ssjamthe/stockpredictiondags/blob/master/Copy_of_StockPredictBasebackup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

In [ ]:
!python --version

Python 2.7.16


In [ ]:
# TODO(ccy): Fix need to use deprecated pip resolver.
#!pip install -q -U --use-deprecated=legacy-resolver tfx


In [ ]:
#!pip install apache-beam

In [ ]:
import sys
TOP_PATH = '/Users/swapnil/work/temp/local_data/threshold_1'
sys.path.append(TOP_PATH + '/files')

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import external_input
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from typing import Optional, Text, List, Dict, Any
import tensorflow_model_analysis as tfma

from ml_metadata.proto import metadata_store_pb2
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer import executor as trainer_executor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.extensions.google_cloud_ai_platform.pusher import executor as ai_platform_pusher_executor
from tfx.extensions.google_cloud_ai_platform.trainer import executor as ai_platform_trainer_executor
from tfx.orchestration import pipeline
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input
from tfx.components.example_gen.import_example_gen.component import ImportExampleGen

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
# TODO(ccy): Revert to `tfx.__version__` after 0.27.0 release.
import tfx.version
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.4.1
TFX version: 0.28.0


In [ ]:
context = InteractiveContext()

In [ ]:
# Brings data into the pipeline or otherwise joins/converts training data.
#example_gen = ImportExampleGen(input=external_input('/content/drive/MyDrive/Stock Prediction/testdata'))
example_gen = ImportExampleGen(input=external_input(TOP_PATH + '/data'))
context.run(example_gen)

Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/ImportExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:31,total_bytes:50883215,xor_checksum:1617391251,sum_checksum:50153872473"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/ImportExampleGen/examples/1


In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "increase_threshold_percentage"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "is_higher_than_threshold"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "last_close"
    value {
      float_list {
        value: 29.049999237060547
      }
    }
  }
  feature {
    key: "next_close"
    value {
      float_list {
        value: 31.450000762939453
      }
    }
  }
  feature {
    key: "sequence_close"
    value {
      float_list {
        value: 25.799999237060547
        value: 24.649999618530273
        value: 25.299999237060547
        value: 26.200000762939453
        value: 26.149999618530273
        value: 26.25
        value: 25.75
        value: 25.600000381469727
        value: 25.25
        value: 25.700000762939453
        value: 27.0
        value: 26.899999618530273
        value: 27.700000762939453
        value: 28.149999618530273
        value: 28.4500007629

In [ ]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
#import tensorflow_data_validation as tfdv
#tfdv.visualize_statistics(statistics_gen.outputs['statistics'])

In [ ]:
schema_gen = SchemaGen(
      statistics=statistics_gen.outputs['statistics'],
      infer_feature_shape=True)
context.run(schema_gen)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'increase_threshold_percentage',FLOAT,required,,-
'is_higher_than_threshold',INT,required,,-
'last_close',FLOAT,required,,-
'next_close',FLOAT,required,,-
'sequence_close',FLOAT,required,,-
'sequence_close_higher_than_threshold',FLOAT,required,,-
'sequence_high',FLOAT,required,,-
'sequence_low',FLOAT,required,,-
'sequence_open',FLOAT,required,,-


In [ ]:
example_validator = ExampleValidator(  # pylint: disable=unused-variable
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import external_input
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [ ]:
_features_file = TOP_PATH + '/files/features.py'

In [ ]:
%%writefile {_features_file}

"""
Define constants here that are common across all models
including features names, label and size of vocabulary.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from typing import Text, List

# At least one feature is needed.

# Name of features which have continuous float values. These features will be
# used as their own values.
DENSE_FLOAT_FEATURE_KEYS = []

# Name of features which have continuous float values. These features will be
# bucketized using `tft.bucketize`, and will be used as categorical features.
BUCKET_FEATURE_KEYS = []
# Number of buckets used by tf.transform for encoding each feature. The length
# of this list should be the same with BUCKET_FEATURE_KEYS.
BUCKET_FEATURE_BUCKET_COUNT = [10]

# Name of features which have categorical values which are mapped to integers.
# These features will be used as categorical features.
CATEGORICAL_FEATURE_KEYS = []
# Number of buckets to use integer numbers as categorical features. The length
# of this list should be the same with CATEGORICAL_FEATURE_KEYS.
CATEGORICAL_FEATURE_MAX_VALUES = [24]

# Name of features which have string values and are mapped to integers.
VOCAB_FEATURE_KEYS = ['stock_code']

# Sequence features.
SEQUENCE_FEATURES_FLOAT = ['sequence_open', 'sequence_high', 'sequence_low', 'sequence_close']

# Sequence features numeric.
SEQUENCE_FEATURES_BOOLEAN = ['sequence_close_higher_than_threshold']

# Uncomment below to add more features to the model
# DENSE_FLOAT_FEATURE_KEYS = ['trip_miles', 'fare', 'trip_seconds']
#
# BUCKET_FEATURE_KEYS = [
#     'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
#     'dropoff_longitude'
# ]
# # Number of buckets used by tf.transform for encoding each feature.
# BUCKET_FEATURE_BUCKET_COUNT = [10, 10, 10, 10]
#
# CATEGORICAL_FEATURE_KEYS = [
#     'trip_start_hour', 'trip_start_day', 'trip_start_month'
# ]
# # Number of buckets to use integer numbers as categorical features.
# CATEGORICAL_FEATURE_MAX_VALUES = [24, 31, 12]
#
#
# VOCAB_FEATURE_KEYS = [
#     'payment_type',
#     'company',
# ]

# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 10

# Keys
LABEL_KEY = 'is_higher_than_threshold'


def transformed_name(key: Text) -> Text:
  """Generate the name of the transformed feature from original name."""
  return key + '_xf'


def vocabulary_name(key: Text) -> Text:
  """Generate the name of the vocabulary feature from original name."""
  return key + '_vocab'


def transformed_names(keys: List[Text]) -> List[Text]:
  """Transform multiple feature names at once."""
  return [transformed_name(key) for key in keys]

Overwriting /Users/swapnil/work/temp/local_data/threshold_1/files/features.py


In [ ]:
_preprocessing_file= TOP_PATH + '/files/preprocessing.py'
print(_preprocessing_file)

/Users/swapnil/work/temp/local_data/threshold_1/files/preprocessing.py


In [ ]:
%%writefile {_preprocessing_file}

"""
This file defines a template for TFX Transform component.
"""

from __future__ import division
from __future__ import print_function

import tensorflow as tf
import tensorflow_transform as tft

import features

def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if isinstance(x, tf.sparse.SparseTensor):
    default_value = '' if x.dtype == tf.string else 0
    dense_tensor = tf.sparse.to_dense(
        tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
        default_value)
  else:
    dense_tensor = x

  return tf.squeeze(dense_tensor, axis=1)


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in features.DENSE_FLOAT_FEATURE_KEYS:
    # Preserve this feature as a dense float, setting nan's to the mean.
    outputs[features.transformed_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]))

  for key in features.VOCAB_FEATURE_KEYS:
    # Build a vocabulary for this feature.
    outputs[features.transformed_name(key)] = tft.compute_and_apply_vocabulary(
        _fill_in_missing(inputs[key]),
        top_k=features.VOCAB_SIZE,
        num_oov_buckets=features.OOV_SIZE)

  for key, num_buckets in zip(features.BUCKET_FEATURE_KEYS,
                              features.BUCKET_FEATURE_BUCKET_COUNT):
    outputs[features.transformed_name(key)] = tft.bucketize(
        _fill_in_missing(inputs[key]),
        num_buckets)

  for key in features.CATEGORICAL_FEATURE_KEYS:
    outputs[features.transformed_name(key)] = _fill_in_missing(inputs[key])

  for key in features.SEQUENCE_FEATURES_FLOAT:
    outputs[features.transformed_name(key)] = tf.sparse.from_dense(inputs[key])

  for key in features.SEQUENCE_FEATURES_BOOLEAN:
    outputs[features.transformed_name(key)] = tf.RaggedTensor.from_tensor(
              inputs[key]).to_sparse()

  # TODO(b/157064428): Support label transformation for Keras.
  # Do not apply label transformation as it will result in wrong evaluation.
  outputs[features.transformed_name(
      features.LABEL_KEY)] = inputs[features.LABEL_KEY]

  return outputs

Overwriting /Users/swapnil/work/temp/local_data/threshold_1/files/preprocessing.py


In [ ]:
import importlib
import preprocessing
import features

importlib.reload(preprocessing)
importlib.reload(features)
transform = Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      module_file=os.path.abspath(_preprocessing_file),
      force_tf_compat_v1=False)
context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: /var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Transform/transform_graph/5/.temp_path/tftransform_tmp/81370ee256414f22bac200ec3d5af878/assets
INFO:tensorflow:Assets written to: /var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Transform/transform_graph/5/.temp_path/tftransform_tmp/07a1bd7789eb46618d6e7c1cc4a479ce/assets


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 5
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
transform.outputs

{'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(artifact: id: 5
type_id: 13
uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Transform/transform_graph/5"
custom_properties {
  key: "name"
  value {
    string_value: "transform_graph"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Transform"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 13
name: "TransformGraph"
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 6
type_id: 5
uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Transform/transformed_examples/5"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties

In [ ]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'transform_fn', 'metadata']

In [ ]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 1 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "is_higher_than_threshold_xf"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "sequence_close_higher_than_threshold_xf"
    value {
      float_list {
        value: 0.0
        value: 0.0
        value: 1.0
        value: 1.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 1.0
        value: 1.0
        value: 0.0
        value: 1.0
        value: 1.0
        value: 1.0
        value: 1.0
        value: 0.0
        value: 0.0
        value: 1.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 1.0
        value: 1.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 1.0
        value: 0.0
        value: 1.0

In [ ]:
_trainer_module_file=TOP_PATH + '/files/model.py'

In [ ]:
%%writefile {_trainer_module_file}

"""
A tf.estimator.experimental.RNNClassifier which uses features
defined in features.py and network parameters defined in constants.py.
"""

from __future__ import division
from __future__ import print_function

from absl import logging
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils
from tensorflow.python.keras.utils import losses_utils
from typing import Text, List
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options

from tensorflow_metadata.proto.v0 import schema_pb2
from tfx.utils import io_utils

import features


HIDDEN_UNITS = [16, 8]

TRAIN_BATCH_SIZE = 40
EVAL_BATCH_SIZE = 40


def _gzip_reader_fn(filenames):
  """Small utility returning a record reader that can read gzip'ed files."""
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


# Tf.Transform considers these features as "raw"
def _get_raw_feature_spec(schema):
  return schema_utils.schema_as_feature_spec(schema).feature_spec


def _build_estimator(config, hidden_units=None, warm_start_from=None):
  """Build an estimator for predicting the tipping behavior of taxi riders.
  Args:
    config: tf.estimator.RunConfig defining the runtime environment for the
      estimator (including model_dir).
    hidden_units: [int], the layer sizes of the DNN (input layer first)
    warm_start_from: Optional directory to warm start from.
  Returns:
    A dict of the following:
      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  real_valued_columns = [
    tf.feature_column.numeric_column(key, shape=())
    for key in features.transformed_names(features.DENSE_FLOAT_FEATURE_KEYS)
  ]

  categorical_columns = []
  for key in features.transformed_names(features.VOCAB_FEATURE_KEYS):
    categorical_columns.append(tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_identity(
            key,
            num_buckets=features.VOCAB_SIZE + features.OOV_SIZE,
            default_value=0)))

  for key, num_buckets in zip(
      features.transformed_names(features.BUCKET_FEATURE_KEYS),
      features.BUCKET_FEATURE_BUCKET_COUNT):
    categorical_columns.append(
        tf.feature_column.categorical_column_with_identity(
            key, num_buckets=num_buckets, default_value=0))

  for key, num_buckets in zip(
      features.transformed_names(features.CATEGORICAL_FEATURE_KEYS),
      features.CATEGORICAL_FEATURE_MAX_VALUES):
    categorical_columns.append(
        tf.feature_column.categorical_column_with_identity(
            key, num_buckets=num_buckets, default_value=0))

  sequence_columns = [
    tf.feature_column.sequence_numeric_column(key)
    for key in features.transformed_names(features.SEQUENCE_FEATURES_FLOAT)
  ] + [
       tf.feature_column.sequence_numeric_column(key)
       for key in features.transformed_names(features.SEQUENCE_FEATURES_BOOLEAN)
  ]



  return tf.estimator.experimental.RNNClassifier(
      sequence_feature_columns=sequence_columns, context_feature_columns=categorical_columns, units=[32, 16],
      cell_type='lstm', rnn_cell_fn=None, return_sequences=False, model_dir=config.model_dir,
      n_classes=2, weight_column=None, label_vocabulary=None, optimizer='Adagrad',
      loss_reduction=losses_utils.ReductionV2.SUM_OVER_BATCH_SIZE,
      sequence_mask='sequence_mask', config=config
  )

  #return tf.estimator.DNNLinearCombinedClassifier(
  #    config=config,
  #    linear_feature_columns=categorical_columns,
  #    dnn_feature_columns=real_valued_columns,
  #    dnn_hidden_units=hidden_units or [100, 70, 50, 25],
  #    warm_start_from=warm_start_from)


def _example_serving_receiver_fn(tf_transform_output, schema):
  """Build the serving in inputs.
  Args:
    tf_transform_output: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    Tensorflow graph which parses examples, applying tf-transform to them.
  """
  raw_feature_spec = _get_raw_feature_spec(schema)
  raw_feature_spec.pop(features.LABEL_KEY)

  raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
      raw_feature_spec, default_batch_size=None)
  serving_input_receiver = raw_input_fn()

  transformed_features = tf_transform_output.transform_raw_features(
      serving_input_receiver.features)

  return tf.estimator.export.ServingInputReceiver(
      transformed_features, serving_input_receiver.receiver_tensors)


def _eval_input_receiver_fn(tf_transform_output, schema):
  """Build everything needed for the tf-model-analysis to run the model.
  Args:
    tf_transform_output: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    EvalInputReceiver function, which contains:
      - Tensorflow graph which parses raw untransformed features, applies the
        tf-transform preprocessing operators.
      - Set of raw, untransformed features.
      - Label against which predictions will be compared.
  """
  # Notice that the inputs are raw features, not transformed features here.
  raw_feature_spec = _get_raw_feature_spec(schema)

  serialized_tf_example = tf.compat.v1.placeholder(
      dtype=tf.string, shape=[None], name='input_example_tensor')

  # Add a parse_example operator to the tensorflow graph, which will parse
  # raw, untransformed, tf examples.
  raw_features = tf.io.parse_example(
      serialized=serialized_tf_example, features=raw_feature_spec)

  # Now that we have our raw examples, process them through the tf-transform
  # function computed during the preprocessing step.
  transformed_features = tf_transform_output.transform_raw_features(
      raw_features)

  # The key name MUST be 'examples'.
  receiver_tensors = {'examples': serialized_tf_example}

  # NOTE: Model is driven by transformed features (since training works on the
  # materialized output of TFT, but slicing will happen on raw features.
  raw_features.update(transformed_features)

  return tfma.export.EvalInputReceiver(
      features=raw_features,
      receiver_tensors=receiver_tensors,
      labels=transformed_features[features.transformed_name(
          features.LABEL_KEY)])


def _input_fn(file_pattern: List[Text],
              data_accessor: DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.
  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch
  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=features.transformed_name(features.LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


def _create_train_and_eval_spec(trainer_fn_args, schema):
  """Build the estimator using the high level API.
  Args:
    trainer_fn_args: Holds args used to train the model as name/value pairs.
    schema: Holds the schema of the training examples.
  Returns:
    A dict of the following:
      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """

  tf_transform_output = tft.TFTransformOutput(trainer_fn_args.transform_output)

  train_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      trainer_fn_args.train_files,
      trainer_fn_args.data_accessor,
      tf_transform_output,
      batch_size=TRAIN_BATCH_SIZE)

  eval_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      trainer_fn_args.eval_files,
      trainer_fn_args.data_accessor,
      tf_transform_output,
      batch_size=EVAL_BATCH_SIZE)

  train_spec = tf.estimator.TrainSpec(  # pylint: disable=g-long-lambda
      train_input_fn,
      max_steps=trainer_fn_args.train_steps)

  serving_receiver_fn = lambda: _example_serving_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_output, schema)

  exporter = tf.estimator.FinalExporter('stock-prediction-threshold-base', serving_receiver_fn)
  eval_spec = tf.estimator.EvalSpec(
      eval_input_fn,
      steps=trainer_fn_args.eval_steps,
      exporters=[exporter],
      name='stock-prediction-threshold-base-eval')

  run_config = tf.estimator.RunConfig(
      save_checkpoints_steps=999, keep_checkpoint_max=1)

  run_config = run_config.replace(model_dir=trainer_fn_args.serving_model_dir)

  estimator = _build_estimator(
      hidden_units=HIDDEN_UNITS, config=run_config)

  # Create an input receiver for TFMA processing
  receiver_fn = lambda: _eval_input_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_output, schema)

  return {
    'estimator': estimator,
    'train_spec': train_spec,
    'eval_spec': eval_spec,
    'eval_input_receiver_fn': receiver_fn
  }


# TFX will call this function
def run_fn(fn_args):
  """Train the model based on given args.
  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  schema = io_utils.parse_pbtxt_file(fn_args.schema_file, schema_pb2.Schema())

  train_and_eval_spec = _create_train_and_eval_spec(fn_args, schema)

  # Train the model
  logging.info('Training model.')
  tf.estimator.train_and_evaluate(train_and_eval_spec['estimator'],
                                  train_and_eval_spec['train_spec'],
                                  train_and_eval_spec['eval_spec'])
  logging.info('Training complete.  Model written to %s',
               fn_args.serving_model_dir)

  # Export an eval savedmodel for TFMA
  # NOTE: When trained in distributed training cluster, eval_savedmodel must be
  # exported only by the chief worker.
  logging.info('Exporting eval_savedmodel for TFMA.')
  tfma.export.export_eval_savedmodel(
      estimator=train_and_eval_spec['estimator'],
      export_dir_base=fn_args.eval_model_dir,
      eval_input_receiver_fn=train_and_eval_spec['eval_input_receiver_fn'])

  logging.info('Exported eval_savedmodel to %s.', fn_args.eval_model_dir)

Overwriting /Users/swapnil/work/temp/local_data/threshold_1/files/model.py


In [ ]:
import importlib
#import model
#importlib.reload(model)

from tfx.proto import trainer_pb2

RUN_FN = 'model.run_fn'

TRAIN_NUM_STEPS = 1000
EVAL_NUM_STEPS = 200

ai_platform_training_args=None

# Change this value according to your use cases.
EVAL_ACCURACY_THRESHOLD = 0.6

# Uses user-provided Python function that implements a model using TF-Learn.
trainer_args = {
      'run_fn': RUN_FN,
      'transformed_examples': transform.outputs['transformed_examples'],
      'schema': schema_gen.outputs['schema'],
      'transform_graph': transform.outputs['transform_graph'],
      'train_args': trainer_pb2.TrainArgs(num_steps=TRAIN_NUM_STEPS),
      'eval_args': trainer_pb2.EvalArgs(num_steps=EVAL_NUM_STEPS),
      'custom_executor_spec':
          executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
  }
if ai_platform_training_args is not None:
    trainer_args.update({
        'custom_executor_spec':
            executor_spec.ExecutorClassSpec(
                ai_platform_trainer_executor.GenericExecutor
            ),
        'custom_config': {
            ai_platform_trainer_executor.TRAINING_ARGS_KEY:
                ai_platform_training_args,
        }
    })
trainer = Trainer(**trainer_args)
context.run(trainer)

INFO:tensorflow:Using config: {'_model_dir': '/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Trainer/model/6/serving_model_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 999, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
I

Exception ignored in: <function CapturableResourceDeleter.__del__ at 0x7f8d8d1fdee0>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 725, in _initialize
    self._stateful_fn._get_concrete_function_internal_garbage_collected(  # pylint: disable=protected-access
  File "/Library/Frameworks/P

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Trainer/model/6/serving_model_dir/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.5347041, step = 0
INFO:tensorflow:global_step/sec: 10.849
INFO:tensorflow:loss = 0.5978511, step = 100 (9.218 sec)
INFO:tensorflow:global_step/sec: 11.4705
INFO:tensorflow:loss = 0.6696116, step = 200 (8.718 sec)
INFO:tensorflow:global_step/sec: 11.6979
INFO:tensorflow:loss = 0

ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 8
        type_id: 16
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Trainer/model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 9
        type_id: 17
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Trainer/model_run/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# Get the URI of the output artifact representing the training logs, which is a directory
model_run_dir = trainer.outputs['model'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_dir}

In [ ]:
print(trainer.outputs['model_run'].get()[0].uri)

/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Trainer/model_run/6


In [ ]:
%reload_ext tensorboard

In [ ]:
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))
context.run(model_resolver)

ExecutionResult(
    component_id: Resolver.latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
eval_accuracy_threshold = 0.6

eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='is_higher_than_threshold')],
      slicing_specs=[tfma.SlicingSpec(),
                     tfma.SlicingSpec(feature_keys=['stock_code_xf'])],
      metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(
                  class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': eval_accuracy_threshold}),
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
          ])
      ],
      )
evaluator = Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      # Change threshold will be ignored if there is no baseline (first run).
      eval_config=eval_config)
context.run(evaluator)

ERROR:absl:There are change thresholds, but the baseline is missing. This is allowed only when rubber stamping (first run).
Exception ignored in: <function CapturableResourceDeleter.__del__ at 0x7f8d8d1fdee0>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 725, in _initialize
    self._stat

ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 10
        type_id: 20
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Evaluator/evaluation/8"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 11
        type_id: 21
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Evaluator/blessing/8"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Trainer/model/6"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 8
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
evaluator.outputs

{'evaluation': Channel(
    type_name: ModelEvaluation
    artifacts: [Artifact(artifact: id: 10
type_id: 20
uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Evaluator/evaluation/8"
custom_properties {
  key: "name"
  value {
    string_value: "evaluation"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Evaluator"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 20
name: "ModelEvaluation"
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'blessing': Channel(
    type_name: ModelBlessing
    artifacts: [Artifact(artifact: id: 11
type_id: 21
uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Evaluator/blessing/8"
custom_properties {
  key: "blessed"
  value {
    int_value: 1
  }
}
custom_properties {
  key: "current_model"
  value {
    string_valu

In [ ]:
context.show(evaluator.outputs['evaluation'])

In [ ]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
#tfma.view.render_slicing_metrics(
#    tfma_result, slicing_column='stock_code_xf')

In [ ]:
serving_model_dir=os.path.abspath('base_mode')
ai_platform_serving_args=None

pusher_args = {
      'model':
          trainer.outputs['model'],
      'model_blessing':
          evaluator.outputs['blessing'],
      'push_destination':
          pusher_pb2.PushDestination(
              filesystem=pusher_pb2.PushDestination.Filesystem(
                  base_directory=serving_model_dir)),
  }
if ai_platform_serving_args is not None:
    pusher_args.update({
        'custom_executor_spec':
            executor_spec.ExecutorClassSpec(ai_platform_pusher_executor.Executor
                                           ),
        'custom_config': {
            ai_platform_pusher_executor.SERVING_ARGS_KEY:
                ai_platform_serving_args
        },
    })
pusher = Pusher(**pusher_args)  # pylint: disable=unused-variable
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 12
        type_id: 23
        uri: "/var/folders/7t/rgpvpkmj6bz5lfjwmd6zxfhm0000gn/T/tfx-interactive-2021-04-09T01_21_31.991470-a8xelagf/Pusher/pushed_model/9"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "/Users/swapnil/work/temp/local_data/threshold_1/data/base_mode/1617911684"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1617911684"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))